In [ ]:
#import pytest
import drjit as dr
import mitsuba as mi
mi.set_variant("cuda_ad_spectral")
#mi.set_variant("llvm_ad_spectral")
#mi.set_variant("scalar_spectral")

[mitsuba] Warning: Couldn't import the ipywidgets package. Installing this package is required for the system to properly log messages and print in Jupyter notebooks!


In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
from osgeo import gdal
import imageio
def saveToHdr_no_transform(npArray, dstFilePath, wlist, output_format):
        dshape = npArray.shape
        if len(dshape) == 3:
            bandnum = dshape[2]
        else:
            bandnum = 1
        if output_format == "ENVI":
            format = "ENVI"
        else:
            format = "GTiff"
            dstFilePath += ".tif"
        driver = gdal.GetDriverByName(format)
        dst_ds = driver.Create(dstFilePath, dshape[1], dshape[0], bandnum, gdal.GDT_Float32)
        #     npArray = linear_stretch_3d(npArray)
        if bandnum > 1:
            for i in range(1, bandnum + 1):
                dst_ds.GetRasterBand(i).WriteArray(npArray[:, :, i - 1])
        else:
            dst_ds.GetRasterBand(1).WriteArray(npArray)
        dst_ds = None

        if output_format == "ENVI" and len(wlist) >0:
            # wirte wavelength
            f = open(dstFilePath+".hdr",'r')
            text = f.read()
            f.close()
            wstr = "\nwavelength = {"
            for i in range(0,len(wlist)):
                wstr += str(wlist[i]) + ","
            wstr = wstr[0:len(wstr)-1] + "}"
            f = open(dstFilePath+".hdr",'w')
            text = text + wstr
            f.write(text)
            f.close()

In [5]:
scene = mi.load_file('./Parameters/inverse_allband.xml')
image_ref = mi.render(scene)
data = image_ref[:,:,:]
np.save('./temp/temp1.npy',data)
data = np.load("./temp/temp1.npy")
bandlist = [490,560,665,865]
saveToHdr_no_transform(data, 'image-bandall', bandlist, 'TIFF')